In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用自定义训练容器和模型的自定义训练

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom_training_container_and_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom_training_container_and_model_registry.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/custom_training_container_and_model_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Vertex AI SDK使用自定义容器进行训练，并自动注册模型。

了解更多关于[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将使用 Vertex AI SDK 在自定义的 Docker 容器中从 Python 脚本创建自定义模型，并自动将该模型注册到 Vertex AI 模型注册表中。您也可以选择使用 `gcloud` 命令行工具或云控制台在线创建自定义模型。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 模型注册表
- Vertex AI 训练

执行的步骤包括：

- 创建一个用于训练模型的 Vertex AI 自定义作业。
- 使用自定义容器训练和注册 TensorFlow 模型。
- 从 Vertex AI 模型注册表中列出已注册的模型。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。你将使用的数据集版本已经内置在TensorFlow中。训练好的模型会预测图像属于十种类别中的哪一种：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、轮船、卡车。

费用

本教程使用 Google Cloud 中的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成费用估算。

## 安装

安装以下必要的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform --quiet

仅限Colab使用：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用的是笔记本环境，下面的步骤都是必须的。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得300美元的免费信用额度，可用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果你不知道你的项目ID**，请尝试以下操作：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的`REGION`变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账号

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

1. 顶点 AI 工作台
* 无需操作，因为您已经经过身份验证。

本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化用于 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 来使用支持 GPU 的容器映像以及分配给虚拟机实例的 GPU 数量。例如，要使用一个包含 4 个 Nvidia Telsa K80 GPU 的 GPU 容器映像分配到每个 VM，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器映像。

在此处了解更多有关您区域的硬件加速器支持信息：[在此处](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

设置预构建的容器

为预测设置预构建的Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。下面列出了一些可用的预构建映像：


要获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2-7"

DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU的内存为3.75GB。
     - `n1-highmem`：每个虚拟CPU的内存为6.5GB。
     - `n1-highcpu`：每个虚拟CPU的内存为0.9GB。
 - `vCPUs`：数量为\ [2, 4, 8, 16, 32, 64, 96 \]

*注意：以下内容不支持用于训练：*

 - `标准`：2个虚拟CPU
 - `高CPU`：2、4和8个虚拟CPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

# 教程

现在你已经准备好开始创建自己的自定义模型并对CIFAR10进行训练。

## 训练模型

您可以使用容器映像的两种方式来训练一个定制模型：

- **使用 Google Cloud 预构建容器**。如果您使用预构建容器，您还需要指定要安装到容器映像中的 Python 软件包。这个 Python 软件包包含了您用于训练定制模型的代码。

- **使用您自己的自定义容器映像**。如果您使用自己的容器，容器需要包含您用于训练定制模型的代码。

### 创建一个Docker文件

在本教程中，您将使用自己的自定义容器训练一个CIFAR10模型。

为了使用自己的自定义容器，您需要创建一个Docker文件。首先，您需要为容器组件创建一个目录。

### 检查培训包

#### 包布局

在开始培训之前，您将看看如何为定制培训工作创建Python包。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`包含了将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行定制培训工作的Python脚本。*注意*，当我们在worker pool规范中提到它时，我们用点(`trainer.task`)替换了目录斜杠，并去掉了文件后缀(`.py`)。

#### 包组装

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您可以编写训练脚本 task.py 的内容。我们不会详细说明，只是让您浏览一下。总结如下：

- 从命令行获取保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF 数据集（tfds）加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 根据参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练后的模型保存（`save(args.model_dir)`）到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

编写Docker文件内容

将您的代码容器化的第一步是创建一个Docker文件。在您的Docker文件中，您将包含运行容器镜像所需的所有命令。它将安装您正在使用的所有库，并设置用于训练代码的入口点。

1. 从TensorFlow仓库中安装预定义的深度学习图像容器。
2. 复制Python训练代码，稍后将显示。
3. 将入口点设置为Python训练脚本的`trainer/task.py`。注意，ENTRYPOINT命令中省略了`.py`，因为它是隐含的。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建本地容器

接下来，您将为您的客户容器提供一个名称，用于将其提交到Google容器注册表。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/cifar10:v1"

接下来，建造容器。

In [ ]:
! docker build custom -t $TRAIN_IMAGE

在本地测试容器

在你的笔记本实例中运行容器，以确保它能正常工作。你将运行5个周期。

In [ ]:
! docker run $TRAIN_IMAGE --epochs=5

注册自定义容器
当您在本地完成运行容器之后，将其推送到Google容器注册表。

In [ ]:
! docker push $TRAIN_IMAGE

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

### 创建并运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练作业，2) 运行该作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类来创建一个自定义训练作业，需要设置以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。
- `model_serving_container_image_uri`：用于部署模型的容器。

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="cifar10",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

print(job)

### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数：

- `args`：传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：为了我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
     - 直接：将Cloud Storage位置作为命令行参数传递给你的训练脚本（设置变量`DIRECT = True`），或者
     - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给你的训练脚本（设置变量`DIRECT = False`）。在这种情况下，你在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的周期数。
  - `"--steps=" + STEPS`：每周期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
]

运行自定义训练作业

接下来，通过调用方法 `run`，使用以下参数运行自定义作业来启动训练作业：

- `args`: 要传递给训练脚本的命令行参数。
- `replica_count`: 用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`: 用于计算实例的机器类型。
- `accelerator_type`: 硬件加速器类型。
- `accelerator_count`: 要连接到工作副本的加速器数量。
- `base_output_dir`: 用于写入模型工件的 Cloud 存储位置。
- `model_display_name`: 注册模型的可读名称。
- `sync`: 是否阻塞直到任务完成。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        model_display_name="cifar10",
        sync=True,
    )
else:
    model = job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        model_display_name="cifar10",
        sync=True,
    )

### 在模型注册表中查看模型

最后，`run()` 方法将返回自动注册在模型注册表中的模型。

In [ ]:
print(model.gca_resource)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

job.delete()
model.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI